<table align="left"><tr><td>
<a href="https://colab.research.google.com/github/rickiepark/nlp-with-transformers/blob/main/01_introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="코랩에서 실행하기"/></a>
</td></tr></table>

In [ ]:
# 코랩을 사용하지 않으면 이 셀의 코드를 주석 처리하세요.
!git clone https://github.com/rickiepark/nlp-with-transformers.git
%cd nlp-with-transformers
from install import *
install_requirements(chapter=1)

Cloning into 'nlp-with-transformers'...
remote: Enumerating objects: 542, done.
remote: Counting objects: 100% (276/276), done.
remote: Compressing objects: 100% (169/169), done.
remote: Total 542 (delta 167), reused 184 (delta 107), pack-reused 266
Receiving objects: 100% (542/542), 46.22 MiB | 13.23 MiB/s, done.
Resolving deltas: 100% (266/266), done.
Updating files: 100% (118/118), done.
/content/nlp-with-transformers
⏳ Installing base requirements ...
✅ Base requirements installed!
Using transformers v4.26.1
Using datasets v2.10.1
Using accelerate v0.16.0
Using sentencepiece v0.1.97
Using sacremoses v0.0.41


## 허깅 페이스 트랜스포머스

## 트랜스포머 애플리케이션 둘러보기

In [3]:
text = """Dear Amazon, last week I ordered an Optimus Prime action figure \
from your online store in Germany. Unfortunately, when I opened the package, \
I discovered to my horror that I had been sent an action figure of Megatron \
instead! As a lifelong enemy of the Decepticons, I hope you can understand my \
dilemma. To resolve the issue, I demand an exchange of Megatron for the \
Optimus Prime figure I ordered. Enclosed are copies of my records concerning \
this purchase. I expect to hear from you soon. Sincerely, Bumblebee."""

### 텍스트 분류(영어)

In [1]:
from transformers import pipeline

classifier = pipeline("text-classification")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [4]:
import pandas as pd

outputs = classifier(text)
pd.DataFrame(outputs)

,label,score
0,NEGATIVE,0.901546


### 텍스트 분류(한국어)

In [5]:
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer

# 한국어 텍스트 분류 모델과 토크나이저 로드
model_name = "kykim/bert-kor-base"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 분류 파이프라인 초기화
classifier = pipeline('text-classification', model=model, tokenizer=tokenizer)

# 텍스트 분류
result = classifier("한국어 자연어 처리는 재미있습니다.")
print(result)

config.json:   0%|          | 0.00/725 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/476M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at kykim/bert-kor-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/344k [00:00<?, ?B/s]

[{'label': 'LABEL_0', 'score': 0.5221877694129944}]


### 개체명 인식(영어)

In [6]:
ner_tagger = pipeline("ner", aggregation_strategy="simple")
outputs = ner_tagger(text)
pd.DataFrame(outputs)

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

,entity_group,score,word,start,end
0,ORG,0.879010,Amazon,5,11
1,MISC,0.990859,Optimus Prime,36,49
2,LOC,0.999755,Germany,90,97
3,MISC,0.556571,Mega,208,212
4,PER,0.590256,##tron,212,216
5,ORG,0.669692,Decept,253,259
6,MISC,0.498349,##icons,259,264
7,MISC,0.775362,Megatron,350,358
8,MISC,0.987854,Optimus Prime,367,380
9,PER,0.812096,Bumblebee,502,511


### 개체명 인식(한국어)

In [3]:
from transformers import AutoModelForTokenClassification, AutoTokenizer, pipeline

# 한국어 개체명 인식 모델과 토크나이저 로드
model_name = "klue/roberta-large"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModelForTokenClassification.from_pretrained(model_name)

# 개체명 인식 파이프라인 초기화
ner_pipeline = pipeline('ner', model=model, tokenizer=tokenizer)

# 개체명 인식 수행
text = "삼성전자는 한국의 대표적인 전자기업이며, 이재용은 삼성전자의 부회장이다."
result = ner_pipeline(text)

# 결과 출력
for entity in result:
    print(f"단어: {entity['word']}, 레이블: {entity['entity']}")

tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.35G [00:00<?, ?B/s]

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


단어: 삼성전자, 레이블: LABEL_1
단어: ##는, 레이블: LABEL_1
단어: 한국, 레이블: LABEL_1
단어: ##의, 레이블: LABEL_0
단어: 대표, 레이블: LABEL_0
단어: ##적인, 레이블: LABEL_0
단어: 전자, 레이블: LABEL_1
단어: ##기업, 레이블: LABEL_1
단어: ##이, 레이블: LABEL_0
단어: ##며, 레이블: LABEL_0
단어: ,, 레이블: LABEL_1
단어: 이재용, 레이블: LABEL_1
단어: ##은, 레이블: LABEL_0
단어: 삼성전자, 레이블: LABEL_1
단어: ##의, 레이블: LABEL_1
단어: 부회장, 레이블: LABEL_1
단어: ##이다, 레이블: LABEL_1
단어: ., 레이블: LABEL_1


### 질문 답변

In [10]:
text = """Dear Amazon, last week I ordered an Optimus Prime action figure \
from your online store in Germany. Unfortunately, when I opened the package, \
I discovered to my horror that I had been sent an action figure of Megatron \
instead! As a lifelong enemy of the Decepticons, I hope you can understand my \
dilemma. To resolve the issue, I demand an exchange of Megatron for the \
Optimus Prime figure I ordered. Enclosed are copies of my records concerning \
this purchase. I expect to hear from you soon. Sincerely, Bumblebee."""

reader = pipeline("question-answering")
question = "What does the customer want?"
outputs = reader(question=question, context=text)
pd.DataFrame([outputs])

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


,score,start,end,answer
0,0.631292,335,358,an exchange of Megatron


### 텍스트 요약(영어)

In [11]:
summarizer = pipeline("summarization")
outputs = summarizer(text, max_length=60, clean_up_tokenization_spaces=True)
print(outputs[0]['summary_text'])

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

 Bumblebee ordered an Optimus Prime action figure from your online store in Germany. Unfortunately, when I opened the package, I discovered to my horror that I had been sent an action figure of Megatron instead. As a lifelong enemy of the Decepticons, I hope you can understand


### 텍스트 요약(한국어)

In [6]:
!pip uninstall transformers
!pip install --no-cache-dir transformers sentencepiece

# 설치 후 런타임 다시 시작해야 함

Found existing installation: transformers 4.35.2
Uninstalling transformers-4.35.2:
  Would remove:
    /usr/local/bin/transformers-cli
    /usr/local/lib/python3.10/dist-packages/transformers-4.35.2.dist-info/*
    /usr/local/lib/python3.10/dist-packages/transformers/*
Proceed (Y/n)? 
  Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 34.2 MB/s eta 0:00:00


In [2]:
# Use a pipeline as a high-level helper
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import pipeline

# Ref: https://huggingface.co/csebuetnlp/mT5_multilingual_XLSum
tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/mT5_multilingual_XLSum", use_fast=False)
model = AutoModelForSeq2SeqLM.from_pretrained("csebuetnlp/mT5_multilingual_XLSum")

# 요약 파이프라인 설정
summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)

# 요약할 텍스트
text = """
    삼성전자는 2023년 1분기에 강력한 성장을 보였습니다. 이는 주로 반도체 부문의 성장에 기인한 것으로,
    글로벌 반도체 시장의 수요 증가가 주요 요인입니다. 삼성전자는 특히 메모리 반도체와 시스템 반도체 분야에서
    높은 성장률을 기록했습니다. 이에 따라, 삼성전자의 시장 점유율은 더욱 확대되었으며, 글로벌 반도체 시장에서의
    경쟁력을 강화하고 있습니다.
"""

# 요약 수행
summary = summarizer(text, max_length=60, min_length=30, do_sample=False)
print(summary[0]['summary_text'])

한국의 삼성전자가 지난 3년 만에 역대 최대 규모의 반도체 제조업체로 선정됐다.


### 기계번역(영어->독일어)

In [4]:
translator = pipeline("translation_en_to_de",
                      model="Helsinki-NLP/opus-mt-en-de")
outputs = translator(text, clean_up_tokenization_spaces=True, min_length=100)
print(outputs[0]['translation_text'])

config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/298M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/768k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/797k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Das ist der Grund, warum ich mich nicht an die Leute klammerte, die in der Stadt leben. Das ist der Grund, warum ich nicht weiß, was ich tun soll. Ich weiß nicht, was ich tun soll, aber ich weiß nicht, was ich tun soll, aber ich weiß nicht, was ich tun soll, aber ich weiß nicht, was ich tun soll, aber ich weiß nicht, was ich tun soll, aber ich weiß nicht, was ich tun soll, aber ich weiß nicht, was ich tun soll, aber ich weiß nicht, was ich tun soll, um das zu tun.


### 기계번역(한국어 -> 영어)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import pipeline

# Ref: https://huggingface.co/jbochi/madlad400-3b-mt
tokenizer = AutoTokenizer.from_pretrained("jbochi/madlad400-3b-mt")
model = AutoModelForSeq2SeqLM.from_pretrained("jbochi/madlad400-3b-mt")

# 모델과 토크나이저 로드
pipe = pipeline("translation", model=model, tokenizer=tokenizer)

# 번역 파이프라인 설정
pipe = pipeline("translation", model=model, tokenizer=tokenizer)

# 번역할 한국어 텍스트
korean_text = "안녕하세요, 오늘 날씨가 참 좋네요."

# 번역 수행
translated_text = pipe(korean_text, src_lang="ko", tgt_lang="en")
print(translated_text[0]['translation_text'])

### 텍스트 생성(영어)

In [1]:
from transformers import set_seed
set_seed(42) # 동일 결과를 재현하기 위해 지정

In [4]:
from transformers import pipeline

text = """Dear Amazon, last week I ordered an Optimus Prime action figure \
from your online store in Germany. Unfortunately, when I opened the package, \
I discovered to my horror that I had been sent an action figure of Megatron \
instead! As a lifelong enemy of the Decepticons, I hope you can understand my \
dilemma. To resolve the issue, I demand an exchange of Megatron for the \
Optimus Prime figure I ordered. Enclosed are copies of my records concerning \
this purchase. I expect to hear from you soon. Sincerely, Bumblebee."""

generator = pipeline("text-generation")
response = "Dear Bumblebee, I am sorry to hear that your order was mixed up."
prompt = text + "\n\nCustomer service response:\n" + response
outputs = generator(prompt, max_length=200)
print(outputs[0]['generated_text'])

No model was supplied, defaulted to gpt2 and revision 6c0e608 (https://huggingface.co/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Dear Amazon, last week I ordered an Optimus Prime action figure from your online store in Germany. Unfortunately, when I opened the package, I discovered to my horror that I had been sent an action figure of Megatron instead! As a lifelong enemy of the Decepticons, I hope you can understand my dilemma. To resolve the issue, I demand an exchange of Megatron for the Optimus Prime figure I ordered. Enclosed are copies of my records concerning this purchase. I expect to hear from you soon. Sincerely, Bumblebee.

Customer service response:
Dear Bumblebee, I am sorry to hear that your order was mixed up. There are a number of possible reasons why it took time for you to review the item. If you see any item that was sent to my address it may not immediately return for a refund. While you may have heard my story in an email, I did not receive any response from you so your mistake could not be corrected and this time


### 텍스트 생성(한국어)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("ai-forever/mGPT")
model = AutoModelForCausalLM.from_pretrained("ai-forever/mGPT")

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)